In [ ]:
import sys
import subprocess
import importlib
import os
import time
import tkinter as tk
from tkinter import filedialog, messagebox

# ---------------------------------------------------------
# 1. 패키지 자동 설치 함수
# ---------------------------------------------------------
def install_package(module_name, package_name=None):
    if package_name is None:
        package_name = module_name
    try:
        importlib.import_module(module_name)
    except ImportError:
        print(f"Installing {package_name} ...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
            print(f"{package_name} installation completed")
        except subprocess.CalledProcessError as e:
            print(f"{package_name} installation failed (exit code {e.returncode})")
            sys.exit(1)

print("Checking required packages...")
install_package("h5py")
install_package("numpy")
install_package("cv2", "opencv-python")
install_package("PIL", "Pillow")
install_package("tensorflow")

import h5py
import cv2
import numpy as np
from PIL import Image, ImageOps, ImageDraw, ImageFont
from tensorflow.keras.models import load_model

# Tkinter 루트 윈도우 생성 (숨김 상태)
root = tk.Tk()
root.withdraw() 

# ---------------------------------------------------------
# 2. 모델 파일 선택 및 로드
# ---------------------------------------------------------
print("Please select the model file (.h5)...")

# 팝업창을 최상단으로 띄우기 설정
root.lift()
root.attributes('-topmost', True)

model_filename = filedialog.askopenfilename(
    title="Select Teachable Machine Model File",
    filetypes=[("H5 Files", "*.h5"), ("All Files", "*.*")]
)

root.attributes('-topmost', False)

if not model_filename:
    print("No model selected. Exiting...")
    sys.exit()

# [모델 호환성 패치] Teachable Machine 버그 수정
print(f"Checking compatibility for: {model_filename}")
try:
    f = h5py.File(model_filename, mode="r+")
    model_config_string = f.attrs.get("model_config")
    
    if isinstance(model_config_string, bytes):
        model_config_string = model_config_string.decode('utf-8')

    if model_config_string.find('"groups": 1,') != -1:
        print("Patching model config (removing 'groups': 1)...")
        model_config_string = model_config_string.replace('"groups": 1,', '')
        f.attrs.modify('model_config', model_config_string)
        f.flush()
        print("Model patched successfully.")
    f.close()
except Exception as e:
    print(f"Warning during model patch: {e}")

# 모델 로드
print("Loading Keras model...")
try:
    model = load_model(model_filename, compile=False)
    print("Model loaded successfully!")
except Exception as e:
    messagebox.showerror("Error", f"Failed to load model:\n{e}")
    sys.exit()

# 클래스 이름 (학습할 때 설정한 순서대로)
# 필요하다면 이 리스트를 수정하거나 labels.txt를 읽어오는 코드를 추가할 수 있습니다.
class_names = ['_0_forward', '_1_right', '_2_left', '_3_stop']

# ---------------------------------------------------------
# 3. 테스트할 이미지 다중 선택
# ---------------------------------------------------------
print("Please select test images...")

root.lift()
root.attributes('-topmost', True)

# askopenfilenames (복수형) 사용 -> 여러 파일 선택 가능
file_paths = filedialog.askopenfilenames(
    title="Select Test Images (Multi-select available)",
    filetypes=[("Images", "*.jpg *.jpeg *.png *.bmp"), ("All Files", "*.*")]
)

root.attributes('-topmost', False)
root.destroy() # 창 닫기

if not file_paths:
    print("No images selected. Exiting...")
    sys.exit()

print(f"\nSelected {len(file_paths)} images.")

# ---------------------------------------------------------
# 4. 추론 및 결과 출력 루프
# ---------------------------------------------------------
# Teachable Machine 입력 크기
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

print("\nPress any key to see the next image (ESC to exit).")

for img_path in file_paths:
    try:
        # 1. 이미지 로드 (PIL)
        image = Image.open(img_path).convert("RGB")
        
        # 2. 전처리 (Resize & Normalize)
        # Teachable Machine은 이미지를 꽉 채우고(fit) 가운데를 자르는(crop) 방식을 사용함
        image_processed = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
        image_array = np.asarray(image_processed)
        
        # 정규화: (0 ~ 255) -> (-1 ~ 1)
        normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1.0
        data[0] = normalized_image_array

        # 3. 추론 (Prediction)
        prediction = model.predict(data, verbose=0)
        idx = np.argmax(prediction)
        confidence = prediction[0][idx] * 100 # 백분율

        # 결과 텍스트
        result_text = f"Pred: {class_names[idx]} ({confidence:.1f}%)"
        file_name = os.path.basename(img_path)
        print(f"[{file_name}] -> {result_text}")

        # 4. 결과 시각화 (OpenCV 사용)
        # 원본 이미지를 OpenCV 포맷으로 변환 (PIL -> CV2)
        open_cv_image = np.array(image) 
        # RGB -> BGR 변환
        open_cv_image = open_cv_image[:, :, ::-1].copy() 
        
        # 보기 좋게 리사이즈 (너무 크면 줄임)
        h, w = open_cv_image.shape[:2]
        if w > 800:
            scale = 800 / w
            open_cv_image = cv2.resize(open_cv_image, (int(w*scale), int(h*scale)))
        
        # 화면에 텍스트 그리기
        # (검은 배경 + 흰 글씨로 잘 보이게 처리)
        cv2.rectangle(open_cv_image, (0, 0), (400, 40), (0, 0, 0), -1) # 상단 검은띠
        cv2.putText(open_cv_image, result_text, (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        cv2.imshow("Result (Press any key for next)", open_cv_image)
        
        # 키 입력 대기
        key = cv2.waitKey(0)
        if key == 27: # ESC 누르면 전체 종료
            print("Exiting...")
            break

    except Exception as e:
        print(f"Error processing {img_path}: {e}")

cv2.destroyAllWindows()

In [ ]:
import sys
import subprocess
import importlib
import os
import time
import struct
import socket
import json # 설정 파일 로드용
import tkinter as tk
from tkinter import simpledialog, filedialog, messagebox

# ---------------------------------------------------------
# 1. 패키지 자동 설치
# ---------------------------------------------------------
def install_package(module_name, package_name=None):
    if package_name is None:
        package_name = module_name
    try:
        importlib.import_module(module_name)
    except ImportError:
        print(f"Installing {package_name} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

print("Checking required packages...")
install_package("h5py")
install_package("numpy")
install_package("cv2", "opencv-python")
install_package("PIL", "Pillow")
install_package("tensorflow")

import h5py
import cv2
import numpy as np
from PIL import Image, ImageOps
import tensorflow as tf
from tensorflow.keras.models import load_model

# ---------------------------------------------------------
# 2. 설정 및 연결
# ---------------------------------------------------------
HOST_CAM = '192.168.0.60' # [주의] 초기 IP
PORT_CAM = 80
PORT_MOT = 81

client_cam = None
client_mot = None

# Tkinter (IP 입력 및 파일 선택용)
root = tk.Tk()
root.withdraw() 

while True:
    try:
        print(f"Connecting to {HOST_CAM}...")
        client_cam = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_mot = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        
        # 반응 속도 최적화
        client_mot.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)
        
        client_cam.settimeout(3)
        client_mot.settimeout(3)
        
        client_cam.connect((HOST_CAM, PORT_CAM))
        client_mot.connect((HOST_CAM, PORT_MOT))
        
        client_cam.settimeout(None)
        client_mot.settimeout(None)
        print("Connected successfully!")
        break 
    except Exception as e:
        print(f"Connection Failed: {e}")
        new_ip = simpledialog.askstring("Connection Failed", 
                                      f"Failed to connect to {HOST_CAM}.\n\nEnter ESP32 IP Address:",
                                      parent=root,
                                      initialvalue=HOST_CAM)
        if new_ip: HOST_CAM = new_ip
        else: sys.exit()

# ---------------------------------------------------------
# 3. 모델 로드 (Teachable Machine 호환성 패치 포함)
# ---------------------------------------------------------
model_filename = 'keras_model.h5'

# 파일 없으면 선택창
if not os.path.exists(model_filename):
    root.lift(); root.attributes('-topmost', True)
    model_filename = filedialog.askopenfilename(title="Select Teachable Machine Model", filetypes=[("H5 Files", "*.h5")])
    root.attributes('-topmost', False)
    
    if not model_filename:
        print("No model selected."); sys.exit()

# 모델 구조 패치 (groups=1 버그 수정)
try:
    f = h5py.File(model_filename, mode="r+")
    cfg = f.attrs.get("model_config")
    if isinstance(cfg, bytes): cfg = cfg.decode('utf-8')
    if '"groups": 1,' in cfg:
        print("Patching model config...")
        f.attrs.modify('model_config', cfg.replace('"groups": 1,', ''))
        f.flush()
    f.close()
except: pass

try:
    print(f"Loading model: {model_filename} ...")
    model = load_model(model_filename, compile=False)
    print("Model loaded.")
except Exception as e:
    messagebox.showerror("Error", f"Failed to load model:\n{e}")
    sys.exit()

root.destroy()

names = ['_0_forward', '_1_right', '_2_left', '_3_stop']
cmd_chars = [b'F', b'R', b'L', b'S'] 

# ---------------------------------------------------------
# 4. 마스킹 설정 로드 및 트랙바 초기화
# ---------------------------------------------------------
CONFIG_FILE = "mask_config.json"
crop_top, crop_bottom, crop_left, crop_right = 0, 0, 0, 0

# 설정 파일 로드
if os.path.exists(CONFIG_FILE):
    try:
        with open(CONFIG_FILE, 'r') as f:
            config = json.load(f)
            crop_top = config.get("top", 0)
            crop_bottom = config.get("bottom", 0)
            crop_left = config.get("left", 0)
            crop_right = config.get("right", 0)
        print(f"Loaded mask config: {config}")
    except:
        print("Failed to load mask config. Using default (0).")

def nothing(x): pass

WINDOW_NAME = 'Teachable Machine Driving (Masking Control)'
cv2.namedWindow(WINDOW_NAME)

# 트랙바 생성
cv2.createTrackbar('Top', WINDOW_NAME, crop_top, 240, nothing)
cv2.createTrackbar('Bottom', WINDOW_NAME, crop_bottom, 240, nothing)
cv2.createTrackbar('Left', WINDOW_NAME, crop_left, 320, nothing)
cv2.createTrackbar('Right', WINDOW_NAME, crop_right, 320, nothing)

# ---------------------------------------------------------
# 5. 주행 루프
# ---------------------------------------------------------
t_prev = time.time()
cnt_frame = 0
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# 데이터 수집 때와 해상도 통일
DISPLAY_WIDTH = 640
DISPLAY_HEIGHT = 480

try:
    while True:
        # 1. 영상 수신
        try:
            client_cam.sendall(struct.pack('B', 12))
            data_len_bytes = client_cam.recv(4)
            if not data_len_bytes: break
            data_len = struct.unpack('I', data_len_bytes)[0]
            
            img_data = b''
            while len(img_data) < data_len:
                packet = client_cam.recv(data_len - len(img_data))
                if not packet: break
                img_data += packet
            if len(img_data) < data_len: break
            
            np_data = np.frombuffer(img_data, dtype='uint8')
            frame = cv2.imdecode(np_data, 1)
            if frame is None: continue

        except Exception:
            print("Connection Error")
            break

        # 2. 리사이즈 (640x480) - 마스킹 기준 해상도
        frame_resized = cv2.resize(frame, (DISPLAY_WIDTH, DISPLAY_HEIGHT))

        # 3. 트랙바 값 읽어서 마스킹 적용
        c_top = cv2.getTrackbarPos('Top', WINDOW_NAME)
        c_bottom = cv2.getTrackbarPos('Bottom', WINDOW_NAME)
        c_left = cv2.getTrackbarPos('Left', WINDOW_NAME)
        c_right = cv2.getTrackbarPos('Right', WINDOW_NAME)

        h, w, _ = frame_resized.shape
        if c_top > 0: frame_resized[:c_top, :] = 0
        if c_bottom > 0: frame_resized[h-c_bottom:, :] = 0
        if c_left > 0: frame_resized[:, :c_left] = 0
        if c_right > 0: frame_resized[:, w-c_right:] = 0

        # 화면 출력
        cv2.imshow(WINDOW_NAME, frame_resized)

        # 4. AI 모델 입력용 전처리 (PIL 변환 -> 224x224 리사이즈 -> 정규화)
        # 중요: 마스킹된 이미지(frame_resized)를 사용해야 함
        image = Image.fromarray(cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB))
        image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
        
        normalized_image_array = (np.asarray(image).astype(np.float32) / 127.5) - 1.0
        data[0] = normalized_image_array

        # 5. 추론
        prediction = model.predict(data, verbose=0)
        idx = np.argmax(prediction)
        
        print(f"Pred: {names[idx]} ({prediction[0][idx]:.2f})")

        # 6. 모터 제어
        try:
            client_mot.sendall(cmd_chars[idx])
        except:
            pass

        key = cv2.waitKey(1)
        if key == 27: break

        cnt_frame += 1
        if time.time() - t_prev > 1.0:
            print(f"FPS: {cnt_frame}")
            t_prev = time.time()
            cnt_frame = 0

except KeyboardInterrupt:
    print("Stopping...")
finally:
    if client_cam: client_cam.close()
    if client_mot: client_mot.close()
    cv2.destroyAllWindows()